In [1]:
import numpy as np
import time
import cv2
import imutils
from imutils.video import FPS
from imutils.video import VideoStream
import torch

In [5]:
model = torch.hub.load('/content/drive/MyDrive/TYOLO/yolov5', 'custom', path='/content/drive/MyDrive/TYOLO/yolov5/runs/train/exp16/weights/best.pt', source='local')

requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...


YOLOv5 🚀 v6.0-37-g620b535 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441.1875MB)




requirements: 1 package updated per /content/drive/MyDrive/TYOLO/yolov5/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients
Adding AutoShape... 


In [6]:
def calculate_distance(y1,y2) :
  # diag = ((x1-x2)**2 + (y1-y2)**2)**(1/2)
  X = 1.0688
  f = 12
  real_h = 1067
  image_h = 480
  obj_h = abs(y1-y2)
  Sensor_h = 5.29 
  D = (f * image_h * real_h) / (obj_h * Sensor_h)
  return str(int(D/1000))

In [7]:
vs = cv2.VideoCapture('/content/drive/MyDrive/TYOLO/Lara_Test.mp4')
writer = None
(W, H) = (None, None)
# try to determine the total number of frames in the video file
try:
	prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
		else cv2.CAP_PROP_FRAME_COUNT
	total = int(vs.get(prop))
	print("[INFO] {} total frames in video".format(total))
# an error occurred while trying to determine the total
# number of frames in the video file
except:
	print("[INFO] could not determine # of frames in video")
	print("[INFO] no approx. completion time can be provided")
	total = -1

[INFO] 540 total frames in video


In [8]:
count = 0
while True:
  # read the next frame from the file
  (grabbed, frame) = vs.read()
  if count==0 :
    f = frame
    count+=1
  # if the frame was not grabbed, then we have reached the end
  # of the stream
  if not grabbed:
    break
  # if the frame dimensions are empty, grab them
  if W is None or H is None:
    (H, W) = frame.shape[:2]

  optimal_frame = np.zeros((frame.shape[0], frame.shape[1],3),dtype=float)
  # print(type(frame[0:int(frame.shape[0]*0.4)][:]))
  optimal_frame[0:int(frame.shape[0]*0.4)][:] = frame[0:int(frame.shape[0]*0.4)][:]
  start = time.time()
  layerOutputs = model(optimal_frame)
  # print(layerOutputs)
  layerOutputs.print()
  # print(len(layerOutputs.xyxy[0]))
  if len(layerOutputs.xyxy[0]) > 0 :
    for i in layerOutputs.xyxy[0] :
      try:
        x1 = i[0]
        y1 = i[1]
        x2 = i[2]
        y2 = i[3]
        dist = calculate_distance(y1,y2)
        text = 'Traffic Light '+ dist + ' m'
        print(text)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255,255,255), 2)
        cv2.putText(frame,text, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2) #Put a label over box.
        # print(dist)
        # cv2.putText(frame,, (10, 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)
      except:
        continue
  end = time.time()


  if writer is None:
    # initialize our video writer
    fourcc = cv2.VideoWriter_fourcc(*"MJPG")
    writer = cv2.VideoWriter("/content/drive/MyDrive/TYOLO/Test3.mp4", fourcc, 30,
      (frame.shape[1], frame.shape[0]), True)
    # some information on processing single frame
    if total > 0:
      elap = (end - start)
      print("[INFO] single frame took {:.4f} seconds".format(elap))
      print("[INFO] estimated total time to finish: {:.4f}".format(
        elap * total))
  # write the output frame to disk
  writer.write(frame)
  # release the file pointers
print("[INFO] cleaning up...")
writer.release()
vs.release()

/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
image 1/1: 480x640 (no detections)
Speed: 23.0ms pre-process, 104.7ms inference, 21.7ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 (no detections)
Speed: 8.4ms pre-process, 33.6ms inference, 0.8ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 (no detections)
Speed: 7.6ms pre-process, 33.1ms inference, 1.0ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 (no detections)
Speed: 11.7ms pre-process, 32.9ms inference, 0.8ms NMS per image at shape (1, 3, 480, 640)


[INFO] single frame took 0.1586 seconds
[INFO] estimated total time to finish: 85.6276


image 1/1: 480x640 (no detections)
Speed: 6.7ms pre-process, 33.9ms inference, 4.0ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 (no detections)
Speed: 5.4ms pre-process, 33.5ms inference, 1.3ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 (no detections)
Speed: 5.9ms pre-process, 32.6ms inference, 0.8ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 (no detections)
Speed: 6.0ms pre-process, 33.1ms inference, 0.8ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 (no detections)
Speed: 6.1ms pre-process, 33.1ms inference, 0.9ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 (no detections)
Speed: 6.1ms pre-process, 33.0ms inference, 1.2ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 (no detections)
Speed: 6.0ms pre-process, 32.9ms inference, 0.9ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 (no detections)
Speed: 15.6ms pre-process, 33.3ms inference, 0.8ms NMS per image at shape (1, 3, 480, 640)

Traffic Light 86 m
Traffic Light 81 m


image 1/1: 480x640 1 go
Speed: 5.4ms pre-process, 33.1ms inference, 2.1ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 (no detections)
Speed: 8.1ms pre-process, 33.0ms inference, 1.1ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 (no detections)
Speed: 5.5ms pre-process, 33.1ms inference, 0.7ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 (no detections)
Speed: 5.2ms pre-process, 33.1ms inference, 2.5ms NMS per image at shape (1, 3, 480, 640)


...

Traffic Light 89 m
Traffic Light 89 m
Traffic Light 88 m
Traffic Light 89 m


image 1/1: 480x640 1 stop
Speed: 6.1ms pre-process, 32.9ms inference, 2.6ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 1 stop
Speed: 6.1ms pre-process, 33.1ms inference, 2.7ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 1 stop
Speed: 5.4ms pre-process, 32.8ms inference, 3.3ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 (no detections)
Speed: 6.0ms pre-process, 32.8ms inference, 0.8ms NMS per image at shape (1, 3, 480, 640)


Traffic Light 90 m
Traffic Light 92 m
Traffic Light 91 m


image 1/1: 480x640 (no detections)
Speed: 6.7ms pre-process, 33.0ms inference, 0.8ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 (no detections)
Speed: 7.2ms pre-process, 32.4ms inference, 1.2ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 (no detections)
Speed: 11.2ms pre-process, 32.8ms inference, 0.8ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 (no detections)
Speed: 6.2ms pre-process, 32.8ms inference, 1.0ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 (no detections)
Speed: 6.3ms pre-process, 33.0ms inference, 0.8ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 (no detections)
Speed: 5.5ms pre-process, 33.1ms inference, 0.8ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 (no detections)
Speed: 5.6ms pre-process, 32.9ms inference, 0.8ms NMS per image at shape (1, 3, 480, 640)
image 1/1: 480x640 (no detections)
Speed: 7.9ms pre-process, 32.7ms inference, 0.8ms NMS per image at shape (1, 3, 480, 640)

[INFO] cleaning up...


In [ ]:
def detect_video()